In [ ]:
# ==================== 模块01：核心配置模块 (config.py) ====================
"""
Enhanced Multi-source Remote Sensing Data Fusion System
核心配置模块 - 系统参数和路径配置

作者: 研究团队
日期: 2024
目的: 集中管理系统配置参数，确保模块间一致性
策略: 7-9月中值合成获得完整空间覆盖
"""

import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ==================== Google Earth Engine 初始化 ====================
def initialize_gee():
    """安全的GEE初始化函数"""
    print("正在初始化 Google Earth Engine...")
    try:
        # 先尝试直接初始化（如果已认证）
        ee.Initialize()
        print("✓ Google Earth Engine 初始化成功")
        return True
    except Exception as e:
        print(f"⚠ 直接初始化失败: {e}")
        print("⚠ 尝试认证并初始化 GEE...")
        try:
            ee.Authenticate()
            ee.Initialize()
            print("✓ Google Earth Engine 认证并初始化成功")
            return True
        except Exception as auth_error:
            print(f"✗ GEE 初始化失败: {auth_error}")
            print("✗ 请检查网络连接和GEE账户权限")
            return False

# 在模块导入时尝试初始化GEE，但不强制要求成功
GEE_INITIALIZED = initialize_gee()

# ==================== 核心时空参数配置 ====================
class TimeConfig:
    """时间配置类 - 7-9月中值合成策略"""
    START_DATE = '2022-07-01'  # 7月开始
    END_DATE = '2022-10-01'    # 9月结束
    TARGET_YEAR = 2022
    SEASON = 'Q3'  # 第三季度
    STRATEGY = 'median_composite'  # 中值合成策略
    
    @classmethod
    def get_time_info(cls):
        return {
            'start_date': cls.START_DATE,
            'end_date': cls.END_DATE,
            'year': cls.TARGET_YEAR,
            'season': cls.SEASON,
            'strategy': cls.STRATEGY,
            'description': '7-9月中值合成获得完整空间覆盖'
        }

class SpatialConfig:
    """空间配置类 - 统一投影和分辨率"""
    # 边界文件路径
    BOUNDARY_PATH = '/Users/hanxu/geemap/bdy.shp'
    
    # 统一目标投影和分辨率 - 30m适配所有数据源
    TARGET_SCALE = 30  # 30米分辨率
    TARGET_CRS = 'EPSG:4326'  # WGS84坐标系
    
    # 处理参数
    TILE_SCALE = 4  # GEE瓦片处理尺度
    MAX_PIXELS = 1e13  # 最大像素数
    
    @classmethod
    def load_boundary(cls):
        """加载研究区边界"""
        if not GEE_INITIALIZED:
            raise RuntimeError("Google Earth Engine 未初始化，无法加载边界数据")
            
        try:
            # 检查文件是否存在
            if not os.path.exists(cls.BOUNDARY_PATH):
                raise FileNotFoundError(f"边界文件不存在: {cls.BOUNDARY_PATH}")
            
            # 检查文件格式
            if not cls.BOUNDARY_PATH.lower().endswith('.shp'):
                raise ValueError(f"边界文件必须是shapefile格式: {cls.BOUNDARY_PATH}")
            
            boundary = geemap.shp_to_ee(cls.BOUNDARY_PATH)
            
            # 验证边界数据有效性
            try:
                bounds = boundary.bounds().getInfo()
                print(f"✓ 边界文件加载成功: {cls.BOUNDARY_PATH}")
                print(f"  边界范围: {bounds}")
                return boundary
            except Exception as e:
                raise ValueError(f"边界数据无效或损坏: {e}")
                
        except Exception as e:
            print(f"✗ 边界文件加载失败: {e}")
            raise
    
    @classmethod
    def get_spatial_info(cls):
        return {
            'boundary_path': cls.BOUNDARY_PATH,
            'target_scale': cls.TARGET_SCALE,
            'target_crs': cls.TARGET_CRS,
            'tile_scale': cls.TILE_SCALE,
            'max_pixels': cls.MAX_PIXELS
        }

class DataSourceConfig:
    """数据源配置类 - 多源数据路径和资产ID"""
    
    # === Google Earth Engine 数据集ID ===
    # Landsat 8 Collection 2 Tier 1 Surface Reflectance
    LANDSAT8_SR = 'LANDSAT/LC08/C02/T1_L2'
    
    # Sentinel-1 SAR GRD
    SENTINEL1_GRD = 'COPERNICUS/S1_GRD'
    
    # MODIS Evapotranspiration 8-Day Global 500m
    MODIS_ET = 'MODIS/061/MOD16A2'
    
    # Climate Hazards Group InfraRed Precipitation with Station data
    CHIRPS_PRECIP = 'UCSB-CHG/CHIRPS/DAILY'
    
    # NASA SRTM Digital Elevation 30m
    SRTM_DEM = 'USGS/SRTMGL1_003'
    
    # HydroSHEDS Flow Accumulation
    HYDROSHEDS_FA = 'WWF/HydroSHEDS/15ACC'
    
    # ESA WorldCover 10m
    ESA_WORLDCOVER = 'ESA/WorldCover/v200'  # 修正：使用v200而不是v100
    
    # === 本地数据路径 ===
    # 土壤盐分实测数据
    SOIL_DATA_PATH = '/Users/hanxu/geemap/material/soil/soil_2022_08.csv'
    
    # 土壤类型数据 (GEE资产)
    SOIL_TYPE_ASSET = 'projects/ee-hanxu0223/assets/Soil_Raster'
    
    @classmethod
    def get_gee_collections(cls):
        """获取所有GEE数据集配置"""
        return {
            'landsat8': cls.LANDSAT8_SR,
            'sentinel1': cls.SENTINEL1_GRD,
            'modis_et': cls.MODIS_ET,
            'chirps': cls.CHIRPS_PRECIP,
            'srtm': cls.SRTM_DEM,
            'hydrosheds': cls.HYDROSHEDS_FA,
            'worldcover': cls.ESA_WORLDCOVER,
            'soil_type': cls.SOIL_TYPE_ASSET
        }
    
    @classmethod
    def get_local_paths(cls):
        """获取本地数据路径配置"""
        return {
            'soil_data': cls.SOIL_DATA_PATH,
            'boundary': SpatialConfig.BOUNDARY_PATH
        }
    
    @classmethod
    def verify_local_files(cls):
        """验证本地文件是否存在且可读"""
        issues = []
        
        # 检查土壤数据文件
        if not os.path.exists(cls.SOIL_DATA_PATH):
            issues.append(f"土壤数据文件不存在: {cls.SOIL_DATA_PATH}")
        else:
            # 检查文件格式
            if not cls.SOIL_DATA_PATH.lower().endswith('.csv'):
                issues.append(f"土壤数据文件必须是CSV格式: {cls.SOIL_DATA_PATH}")
            else:
                # 尝试读取文件头部验证格式
                try:
                    test_df = pd.read_csv(cls.SOIL_DATA_PATH, nrows=1)
                    required_columns = ['salinity', 'longitude', 'latitude']
                    missing_cols = [col for col in required_columns if col not in test_df.columns]
                    if missing_cols:
                        issues.append(f"土壤数据文件缺少必需列: {missing_cols}")
                    
                    # 检查数据类型
                    for col in required_columns:
                        if col in test_df.columns:
                            if not pd.api.types.is_numeric_dtype(test_df[col]):
                                issues.append(f"列 '{col}' 应为数值类型")
                                
                except UnicodeDecodeError:
                    issues.append(f"土壤数据文件编码错误，请使用UTF-8编码")
                except pd.errors.EmptyDataError:
                    issues.append(f"土壤数据文件为空")
                except Exception as e:
                    issues.append(f"土壤数据文件格式错误: {e}")
        
        # 检查边界文件
        if not os.path.exists(SpatialConfig.BOUNDARY_PATH):
            issues.append(f"边界文件不存在: {SpatialConfig.BOUNDARY_PATH}")
        else:
            if not SpatialConfig.BOUNDARY_PATH.lower().endswith('.shp'):
                issues.append(f"边界文件必须是shapefile格式: {SpatialConfig.BOUNDARY_PATH}")
            else:
                # 检查shapefile相关文件
                base_path = SpatialConfig.BOUNDARY_PATH[:-4]  # 去掉.shp
                required_files = ['.shx', '.dbf']
                for ext in required_files:
                    if not os.path.exists(base_path + ext):
                        issues.append(f"缺少shapefile组件: {base_path + ext}")
        
        return issues

class OutputConfig:
    """输出配置类 - 文件路径和命名规则"""
    
    # 输出目录
    OUTPUT_DIR = '/Users/hanxu/geemap/out_plots'
    
    # 主要输出文件
    OUTPUT_CSV = 'multi_source_features_median_composite_2022.csv'
    FINAL_CSV = 'final_selected_features_enhanced.csv'
    
    # 验证报告
    VALIDATION_REPORT = 'validation_report.csv'
    FEATURE_IMPORTANCE_REPORT = 'feature_importance_report.csv'
    
    # 可视化文件命名
    VIZ_FILES = {
        'feature_correlation': 'feature_correlation_with_salinity.png',
        'correlation_heatmap': 'feature_correlation_heatmap.png',
        'data_source_contribution': 'data_source_contribution.png',
        'spatial_distribution': 'spatial_distribution_analysis.png',
        'feature_importance_heatmap': 'feature_importance_heatmap.png',
        'feature_selection_curve': 'feature_selection_curve.png',
        'feature_source_distribution': 'feature_source_distribution.png',
        'feature_vs_salinity_scatter': 'feature_vs_salinity_scatter.png'
    }
    
    @classmethod
    def setup_output_directory(cls):
        """创建输出目录"""
        os.makedirs(cls.OUTPUT_DIR, exist_ok=True)
        print(f"✓ 输出目录已准备: {cls.OUTPUT_DIR}")
    
    @classmethod
    def get_output_paths(cls):
        """获取所有输出路径"""
        return {
            'output_dir': cls.OUTPUT_DIR,
            'main_csv': os.path.join(cls.OUTPUT_DIR, cls.OUTPUT_CSV),
            'final_csv': os.path.join(cls.OUTPUT_DIR, cls.FINAL_CSV),
            'validation_report': os.path.join(cls.OUTPUT_DIR, cls.VALIDATION_REPORT),
            'feature_report': os.path.join(cls.OUTPUT_DIR, cls.FEATURE_IMPORTANCE_REPORT)
        }
    
    @classmethod
    def get_viz_path(cls, viz_name):
        """获取可视化文件路径"""
        if viz_name in cls.VIZ_FILES:
            return os.path.join(cls.OUTPUT_DIR, cls.VIZ_FILES[viz_name])
        else:
            return os.path.join(cls.OUTPUT_DIR, f"{viz_name}.png")

class ProcessingConfig:
    """处理配置类 - 算法参数和阈值"""
    
    # === 云掩膜参数 ===
    CLOUD_COVER_THRESHOLD = 70  # 云覆盖阈值 (%)
    CLOUD_COVER_RELAXED = 85    # 放宽的云覆盖阈值 (%)
    
    # === Sentinel-1 参数 ===
    S1_INSTRUMENT_MODE = 'IW'    # 干涉宽幅模式
    S1_ORBIT_PASS = 'ASCENDING'  # 上升轨道
    S1_SPECKLE_FILTER_RADIUS = 50  # 斑点滤波半径 (米)
    
    # === 特征选择参数 ===
    MIN_FEATURES = 15  # 最少特征数
    MAX_FEATURES = 35  # 最多特征数
    
    # === 验证参数 ===
    VALIDATION_WEIGHTS = {
        'ecological': 0.35,
        'physical': 0.30,
        'spatial': 0.20,
        'composite': 0.15
    }
    
    # === 数据源最少特征数 ===
    MIN_FEATURES_PER_SOURCE = {
        'Landsat': 8,
        'Sentinel-1': 3,
        'Environmental': 3,
        'Soil Type': 1,
        'Enhanced': 2
    }
    
    @classmethod
    def get_processing_params(cls):
        """获取所有处理参数"""
        return {
            'cloud_threshold': cls.CLOUD_COVER_THRESHOLD,
            'cloud_relaxed': cls.CLOUD_COVER_RELAXED,
            's1_mode': cls.S1_INSTRUMENT_MODE,
            's1_pass': cls.S1_ORBIT_PASS,
            's1_filter': cls.S1_SPECKLE_FILTER_RADIUS,
            'min_features': cls.MIN_FEATURES,
            'max_features': cls.MAX_FEATURES,
            'validation_weights': cls.VALIDATION_WEIGHTS,
            'min_per_source': cls.MIN_FEATURES_PER_SOURCE
        }

# ==================== 全局配置实例化 ====================
def initialize_system_config():
    """初始化系统配置"""
    print("\n" + "="*70)
    print("增强多源遥感数据融合系统 - 配置初始化")
    print("Enhanced Multi-source Remote Sensing Data Fusion System")
    print("="*70)
    
    # 检查GEE状态
    if not GEE_INITIALIZED:
        print("⚠ 警告: Google Earth Engine 未成功初始化")
        print("  某些功能可能不可用，但配置初始化将继续")
    
    # 创建输出目录
    OutputConfig.setup_output_directory()
    
    # 获取配置信息（不加载边界数据，避免循环依赖）
    time_info = TimeConfig.get_time_info()
    spatial_info = SpatialConfig.get_spatial_info()
    data_sources = DataSourceConfig.get_gee_collections()
    processing_params = ProcessingConfig.get_processing_params()
    
    # 打印配置摘要
    print(f"\n📅 时间配置:")
    print(f"   时间范围: {time_info['start_date']} 至 {time_info['end_date']}")
    print(f"   处理策略: {time_info['description']}")
    
    print(f"\n🌍 空间配置:")
    print(f"   分辨率: {spatial_info['target_scale']} 米")
    print(f"   坐标系: {spatial_info['target_crs']}")
    print(f"   边界文件: {spatial_info['boundary_path']}")
    
    print(f"\n📊 数据源配置:")
    print(f"   GEE数据集: {len(data_sources)} 个")
    print(f"   本地数据: {len(DataSourceConfig.get_local_paths())} 个")
    
    print(f"\n⚙️ 处理参数:")
    print(f"   云覆盖阈值: {processing_params['cloud_threshold']}%")
    print(f"   特征选择范围: {processing_params['min_features']}-{processing_params['max_features']}")
    
    print(f"\n📁 输出配置:")
    print(f"   输出目录: {OutputConfig.OUTPUT_DIR}")
    print(f"   可视化文件: {len(OutputConfig.VIZ_FILES)} 类")
    
    print("\n✓ 系统配置初始化完成!")
    
    return {
        'time_config': time_info,
        'spatial_config': spatial_info,
        'data_sources': data_sources,
        'processing_params': processing_params,
        'output_paths': OutputConfig.get_output_paths(),
        'gee_status': GEE_INITIALIZED
    }

# ==================== 配置验证函数 ====================
def verify_system_config():
    """验证系统配置的有效性"""
    print("\n正在验证系统配置...")
    
    issues = []
    
    # 0. 检查GEE初始化状态
    if not GEE_INITIALIZED:
        issues.append("Google Earth Engine 未成功初始化")
    
    # 1. 检查本地文件
    file_issues = DataSourceConfig.verify_local_files()
    issues.extend(file_issues)
    
    # 2. 检查日期格式和逻辑
    try:
        start_dt = datetime.strptime(TimeConfig.START_DATE, '%Y-%m-%d')
        end_dt = datetime.strptime(TimeConfig.END_DATE, '%Y-%m-%d')
        
        if start_dt >= end_dt:
            issues.append(f"开始日期应早于结束日期: {TimeConfig.START_DATE} vs {TimeConfig.END_DATE}")
        
        # 检查日期是否过于久远或未来
        current_year = datetime.now().year
        if start_dt.year < 1999:  # Landsat 7 开始时间
            issues.append(f"开始年份过早，Landsat数据不可用: {start_dt.year}")
        if end_dt.year > current_year + 1:
            issues.append(f"结束年份过于未来: {end_dt.year}")
        
        # 检查时间跨度是否合理
        time_span = (end_dt - start_dt).days
        if time_span < 30:
            issues.append(f"时间跨度过短({time_span}天)，建议至少30天以获得足够的影像")
        if time_span > 365:
            issues.append(f"时间跨度过长({time_span}天)，可能影响季节一致性")
            
    except ValueError as e:
        issues.append(f"日期格式错误，应为 YYYY-MM-DD: {e}")
    
    # 3. 检查参数合理性
    if ProcessingConfig.MIN_FEATURES >= ProcessingConfig.MAX_FEATURES:
        issues.append(f"最小特征数({ProcessingConfig.MIN_FEATURES})应小于最大特征数({ProcessingConfig.MAX_FEATURES})")
    
    if ProcessingConfig.MIN_FEATURES < 1:
        issues.append(f"最小特征数应大于0: {ProcessingConfig.MIN_FEATURES}")
    
    if ProcessingConfig.MAX_FEATURES > 100:
        issues.append(f"最大特征数过大，建议小于100: {ProcessingConfig.MAX_FEATURES}")
    
    # 4. 检查云覆盖阈值
    for threshold in [ProcessingConfig.CLOUD_COVER_THRESHOLD, ProcessingConfig.CLOUD_COVER_RELAXED]:
        if threshold > 100 or threshold < 0:
            issues.append(f"云覆盖阈值应在0-100之间: {threshold}")
    
    if ProcessingConfig.CLOUD_COVER_THRESHOLD > ProcessingConfig.CLOUD_COVER_RELAXED:
        issues.append("标准云覆盖阈值不应大于放宽阈值")
    
    # 5. 检查空间参数
    if SpatialConfig.TARGET_SCALE <= 0:
        issues.append(f"目标分辨率应大于0: {SpatialConfig.TARGET_SCALE}")
    
    if SpatialConfig.TARGET_SCALE < 10 or SpatialConfig.TARGET_SCALE > 1000:
        issues.append(f"目标分辨率可能不合理(建议10-1000米): {SpatialConfig.TARGET_SCALE}")
    
    # 6. 检查验证权重
    weights = ProcessingConfig.VALIDATION_WEIGHTS
    weight_sum = sum(weights.values())
    if abs(weight_sum - 1.0) > 0.01:
        issues.append(f"验证权重总和应为1.0，当前为: {weight_sum:.3f}")
    
    for key, weight in weights.items():
        if weight < 0 or weight > 1:
            issues.append(f"验证权重应在0-1之间: {key}={weight}")
    
    # 7. 检查每个数据源的最小特征数
    min_per_source = ProcessingConfig.MIN_FEATURES_PER_SOURCE
    total_min = sum(min_per_source.values())
    if total_min > ProcessingConfig.MAX_FEATURES:
        issues.append(f"各数据源最小特征数总和({total_min})超过最大特征数({ProcessingConfig.MAX_FEATURES})")
    
    # 8. 检查输出目录权限
    try:
        os.makedirs(OutputConfig.OUTPUT_DIR, exist_ok=True)
        # 测试写入权限
        test_file = os.path.join(OutputConfig.OUTPUT_DIR, '.test_write_permission')
        with open(test_file, 'w') as f:
            f.write('test')
        os.remove(test_file)
    except PermissionError:
        issues.append(f"输出目录无写入权限: {OutputConfig.OUTPUT_DIR}")
    except Exception as e:
        issues.append(f"输出目录访问失败: {OutputConfig.OUTPUT_DIR} - {e}")
    
    # 9. 检查重要库的可用性
    required_packages = {
        'sklearn': 'scikit-learn',
        'scipy': 'scipy', 
        'geemap': 'geemap',
        'ee': 'earthengine-api'
    }
    
    for module_name, package_name in required_packages.items():
        try:
            __import__(module_name)
        except ImportError:
            issues.append(f"缺少必需的Python包: {package_name} (pip install {package_name})")
    
    # 10. 检查Python版本
    import sys
    python_version = sys.version_info
    if python_version.major < 3 or (python_version.major == 3 and python_version.minor < 7):
        issues.append(f"Python版本过低: {python_version.major}.{python_version.minor}，建议使用Python 3.7+")
    
    # 11. 内存检查（粗略估计）
    try:
        import psutil
        available_memory = psutil.virtual_memory().available / (1024**3)  # GB
        if available_memory < 4:
            issues.append(f"可用内存不足: {available_memory:.1f}GB，建议至少4GB用于大数据处理")
    except ImportError:
        pass  # psutil是可选的
    
    # 报告结果
    if issues:
        print("⚠ 发现配置问题:")
        for i, issue in enumerate(issues, 1):
            print(f"   {i}. {issue}")
        return False
    else:
        print("✓ 系统配置验证通过!")
        return True

# ==================== 全局变量导出 (供其他模块使用) ====================
# 先定义全局变量（避免模块导入时自动执行）
START_DATE = TimeConfig.START_DATE
END_DATE = TimeConfig.END_DATE
TARGET_SCALE = SpatialConfig.TARGET_SCALE
TARGET_CRS = SpatialConfig.TARGET_CRS
OUTPUT_DIR = OutputConfig.OUTPUT_DIR
OUTPUT_CSV = OutputConfig.OUTPUT_CSV
SOIL_DATA_PATH = DataSourceConfig.SOIL_DATA_PATH
SOIL_TYPE_ASSET = DataSourceConfig.SOIL_TYPE_ASSET

# 全局变量，将在主执行时初始化
bdy = None
SYSTEM_CONFIG = None
CONFIG_VALID = False

def run_initialization():
    """运行完整的系统初始化 - 仅在需要时调用"""
    global bdy, SYSTEM_CONFIG, CONFIG_VALID
    
    # 避免重复初始化
    if CONFIG_VALID and bdy is not None:
        print("✓ 系统已初始化，跳过重复初始化")
        return True
    
    try:
        print("\n" + "="*70)
        print("正在执行系统完整初始化...")
        print("="*70)
        
        # 首先检查GEE状态
        if not GEE_INITIALIZED:
            print("❌ Google Earth Engine 未初始化")
            print("  请确保:")
            print("  1. 网络连接正常")
            print("  2. GEE账户已认证 (运行 ee.Authenticate())")
            print("  3. 有足够的GEE配额")
            return False
        
        # 执行基础配置初始化
        SYSTEM_CONFIG = initialize_system_config()
        
        # 验证配置
        print("\n正在验证系统配置...")
        CONFIG_VALID = verify_system_config()
        
        if not CONFIG_VALID:
            print("❌ 配置验证失败，请修复上述问题后重试")
            return False
        
        # 加载边界数据（最后执行，避免重复加载）
        print("\n正在加载边界数据...")
        bdy = SpatialConfig.load_boundary()
        
        print(f"\n🎯 系统完整初始化成功!")
        print(f"📋 配置状态: ✓")
        print(f"📋 边界数据: ✓")
        print(f"📋 GEE状态: ✓")
        print(f"📋 所有组件就绪，可以开始数据处理")
        
        return True
        
    except Exception as e:
        print(f"\n❌ 系统初始化失败: {e}")
        print("请检查以下可能的问题:")
        print("1. 网络连接和GEE服务状态")
        print("2. 文件路径的正确性")
        print("3. 文件格式和完整性")
        print("4. 输出目录的写入权限")
        
        # 重置状态
        SYSTEM_CONFIG = None
        CONFIG_VALID = False
        bdy = None
        
        # 打印详细错误信息（仅在调试模式下）
        import traceback
        print("\n详细错误信息:")
        traceback.print_exc()
        
        return False

# ==================== 快速状态检查函数 ====================
def check_system_status():
    """快速检查系统状态，供其他模块调用"""
    status = {
        'gee_initialized': GEE_INITIALIZED,
        'config_valid': CONFIG_VALID,
        'boundary_loaded': bdy is not None,
        'output_dir_ready': os.path.exists(OUTPUT_DIR),
        'files_accessible': True,
        'ready_for_processing': False
    }
    
    # 检查关键文件是否可访问
    try:
        # 检查边界文件
        if not os.access(SpatialConfig.BOUNDARY_PATH, os.R_OK):
            status['files_accessible'] = False
        
        # 检查土壤数据文件
        if not os.access(DataSourceConfig.SOIL_DATA_PATH, os.R_OK):
            status['files_accessible'] = False
    
    except Exception:
        status['files_accessible'] = False
    
    # 综合状态
    status['ready_for_processing'] = all([
        status['gee_initialized'],
        status['config_valid'], 
        status['boundary_loaded'],
        status['output_dir_ready'],
        status['files_accessible']
    ])
    
    return status

def print_system_status():
    """打印系统状态摘要"""
    status = check_system_status()
    
    print("\n" + "="*50)
    print("系统状态检查")
    print("="*50)
    print(f"GEE初始化: {'✓' if status['gee_initialized'] else '✗'}")
    print(f"配置验证: {'✓' if status['config_valid'] else '✗'}")
    print(f"边界数据: {'✓' if status['boundary_loaded'] else '✗'}")
    print(f"输出目录: {'✓' if status['output_dir_ready'] else '✗'}")
    print(f"文件访问: {'✓' if status['files_accessible'] else '✗'}")
    print("-"*50)
    print(f"处理就绪: {'✓ 可以开始处理' if status['ready_for_processing'] else '✗ 需要初始化'}")
    
    if not status['ready_for_processing']:
        print("\n建议操作:")
        if not status['gee_initialized']:
            print("- 检查网络连接和GEE认证: ee.Authenticate()")
        if not status['config_valid'] or not status['boundary_loaded']:
            print("- 运行 run_initialization() 完成初始化")
        if not status['files_accessible']:
            print("- 检查文件路径和读取权限")
        print("- 运行独立模式: python config.py")
    
    return status
if __name__ == "__main__":
    print("执行配置模块...")
    
    # 运行完整初始化
    success = run_initialization()
    
    if success and CONFIG_VALID:
        print(f"\n✅ 配置模块独立运行成功!")
        print(f"📊 可用配置:")
        print(f"   - 时间范围: {START_DATE} 至 {END_DATE}")
        print(f"   - 空间分辨率: {TARGET_SCALE}m")
        print(f"   - 输出目录: {OUTPUT_DIR}")
        print(f"   - 边界数据: {'已加载' if bdy is not None else '未加载'}")
        print(f"\n📋 下一步: 执行 Landsat 8 处理模块")
    else:
        print(f"\n❌ 配置验证失败，请检查配置参数")

# ==================== 模块导出 ====================
__all__ = [
    # 配置类
    'TimeConfig',
    'SpatialConfig', 
    'DataSourceConfig',
    'OutputConfig',
    'ProcessingConfig',
    # 初始化和状态函数
    'initialize_system_config',
    'verify_system_config',
    'run_initialization',
    'initialize_gee',
    'check_system_status',      # 新增：状态检查
    'print_system_status',      # 新增：状态打印
    # 全局变量 (供其他模块直接使用)
    'START_DATE',
    'END_DATE', 
    'TARGET_SCALE',
    'TARGET_CRS',
    'OUTPUT_DIR',
    'OUTPUT_CSV',
    'SOIL_DATA_PATH',
    'SOIL_TYPE_ASSET',
    'bdy',                      # 边界数据
    'SYSTEM_CONFIG',
    'CONFIG_VALID',
    'GEE_INITIALIZED'
]

# ==================== 模块使用说明 ====================
"""
模块使用方法:

1. 独立运行 (推荐首次使用):
   python config.py
   # 会执行完整的系统初始化和验证

2. 其他模块导入使用:
   from config import (START_DATE, END_DATE, bdy, OUTPUT_DIR, 
                      run_initialization, check_system_status)
   
   # 检查系统状态
   status = check_system_status()
   if not status['ready_for_processing']:
       print("系统未就绪，正在初始化...")
       success = run_initialization()
       if not success:
           raise RuntimeError("系统初始化失败")

3. 快速状态检查:
   from config import print_system_status
   print_system_status()

4. 高级用法:
   from config import TimeConfig, SpatialConfig
   time_info = TimeConfig.get_time_info()

5. 故障排除:
   from config import verify_system_config, GEE_INITIALIZED
   if not GEE_INITIALIZED:
       print("GEE未初始化，请检查网络和认证")
   if not verify_system_config():
       print("请修复配置问题后重试")

注意事项:
- ⚠️ 首次使用建议独立运行此模块进行完整初始化
- ⚠️ 确保GEE账户已认证且有网络连接  
- ⚠️ 检查所有文件路径的正确性
- ⚠️ 验证输出目录的写入权限
- ⚠️ 安装必需的Python库: sklearn, scipy, geemap等
- ✅ 使用 check_system_status() 快速诊断问题
- ✅ 系统支持增量初始化，避免重复操作
"""